# LDA with Collapsed Gibbs Sampling


## train & test データロード

In [1]:
def load_train_test():
    """
    @return train list 学習用の文書集合
    @return test list テスト用の文書集合
    """
    read_dir = './data/ldcourpas/'
    train_doc_name = 'train_doclist.list'
    test_doc_name = 'test_doclist.list'
    
    with open(read_dir + train_doc_name, mode='rb') as f:
        train = pickle.load(f)
    with open(read_dir + test_doc_name, mode='rb') as f:
        test = pickle.load(f)
    
    return train, test

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pickle

train, test = load_train_test()
analyzer = lambda words: words
vect = CountVectorizer(max_features=10000, min_df=.01, max_df=.40, analyzer=analyzer)
X = vect.fit_transform(train)
id2word = {v: k for k, v in vect.vocabulary_.items()}

## LDA

In [2]:
import logging
import scipy.special as special
from sklearn import preprocessing
import time

class LDA(object):
    PRINT_EVERY = 20
    def __init__(self, n_iter, n_topics, logger):
        self.n_iter = n_iter
        self.n_topics = n_topics
        self.topics = np.arange(self.n_topics)
        self.alpha = None
        self.beta = None
        self.logger = logger
        
    def __init_dirichlet_param(self, n_features):
        self.alpha = np.ones(self.n_topics) * 10 / self.n_topics
        self.beta = np.ones(n_features) * 100 / n_features
        self.sum_beta = self.beta.sum()
    
    def __init_latent_vars(self, n_docs, n_features):
        self.z = np.random.randint(0, self.n_topics,
                                   size=(n_docs, n_features))
    
    def __initialize(self, n_docs, n_features, X):
        indices = X.indices
        indptr = X.indptr
        data = X.data
        
        self.ndz = np.zeros((n_docs, self.n_topics))
        self.nzv = np.zeros((self.n_topics, n_features))
        self.nd = np.zeros(n_docs)
        self.nz = np.zeros(self.n_topics)
        self.z = {}
        
        for d in range(n_docs):
                vs = indices[indptr[d]:indptr[d+1]]
                n_vs = data[indptr[d]:indptr[d+1]]
                for v, n_v in zip(vs, n_vs):
                    z = np.random.randint(self.n_topics)
                    self.ndz[d, z] += n_v
                    self.nzv[z, v] += n_v
                    self.nd[d] += n_v
                    self.nz[z] += n_v
                    self.z[(d, v)] = z
    
    def __conditional_distribution(self, d, v):
        nume = (self.ndz[d]+self.alpha) * (self.nzv[:, v]+self.beta[v])
        # denom = (self.nd[d]+sum_alpha) * (self.nz+sum_beta)
        denom = self.nz + self.sum_beta
        p = nume / denom
        p_z = p / p.sum()
        return p_z
    
    def __sampling_z(self, p_z):
        """
        @param p_z ndarray 潜在変数zをサンプリングする多項分布のパラメータ
        @return z int サンプリングした潜在変数zの値
        """
        #return np.random.choice(self.n_topics, p=p_normalized[0], size=1)
        return np.random.multinomial(1, pvals=p_z).argmax()
    
    def __update_dirichlet_param(self, ave_ndz, n_docs):
        """
        update alpha
        
        @param ave_ndz ndarray ndzのサンプル平均
        """
        
        sum_alpha = self.alpha.sum()
        
        ave_ndz_T = ave_ndz.T
        alpha_T = self.alpha.reshape(-1, 1)
        
        nume = (special.psi(ave_ndz_T+alpha_T).sum(axis=1)\
                - n_docs*special.psi(self.alpha))\
                * self.alpha
        
        denom = (special.psi(ave_ndz_T.sum(axis=1)+sum_alpha)\
                - special.psi(sum_alpha))\
                * n_docs
            
        self.alpha = nume / denom
        
        """
        # alphaのupdate
        sum_alpha = self.alpha.sum()
        for k in range(self.n_topics):
            alpha_k = self.alpha[k]
            
            akd = (special.psi(ave_ndz[:,k]+self.alpha[k])\
                    - special.psi(self.alpha[k]))\
                    * self.alpha[k]

            bd = (special.psi(ave_ndz[:,k].sum()+sum_alpha)\
                    - special.psi(sum_alpha))\
                    * float(n_docs)
            
            self.alpha[k] = akd.sum() / bd
        """
    
    """
    # 遅いバージョン
    def __update_dirichlet_param(self, ave_ndz, n_docs):
        alpha_sum = np.sum(self.alpha)
        
        # alphaのupdate
        for k in range(self.n_topics):
            alpha_k = self.alpha[k]
            akd = [(special.psi(ave_ndz[d, k] + alpha_k)\
                      - special.psi(alpha_k)) * alpha_k
                     for d in range(n_docs)]
            bd = [special.psi(np.sum(ave_ndz[:,k]) + alpha_sum)\
                   - special.psi(alpha_sum)
                   for _ in range(n_docs)]
            self.alpha[k] = np.sum(akd) / np.sum(bd)
    """
    
    def fit(self, X, X_test=None):
        """
        @param X scipy.sparse 文書集合
        @return self
        """
        
        n_docs, n_features = X.shape
        indices = X.indices
        indptr = X.indptr
        data = X.data
        
        # initialize params
        self.__init_dirichlet_param(n_features)
        self.__initialize(n_docs, n_features, X)
        
        self.logger.info('ndz: {}, nzv: {}'.format(self.ndz.shape,
                                                     self.nzv.shape))
        
        # ndzのこれまでのサンプル合計
        sum_ndz = self.ndz.copy()
        
        self.logger.info('iteration start!')
        for s in range(self.n_iter):
            start = time.time()
            #ds = np.arange(n_docs)
            #np.random.shuffle(ds)
            for d in range(n_docs):
                vs = indices[indptr[d]:indptr[d+1]]
                n_vs = data[indptr[d]:indptr[d+1]]
                for v, n_v in zip(vs, n_vs):
                # for v in vs:
                    # 文書dのi番目を表すiとvはbag-of-words表現なので同じ
                    current_z = self.z[(d, v)]
                    # z_diを新しくサンプリングするので、現在のz_diに関する個数を引く
                    self.ndz[d, current_z] -= n_v
                    self.nzv[current_z, v] -= n_v
                    self.nz[current_z] -= n_v
                    
                    p_z = self.__conditional_distribution(d, v)
                    z = self.__sampling_z(p_z)
                    
                    self.ndz[d, z] += n_v
                    self.nzv[z, v] += n_v
                    self.nz[z] += n_v
                    self.z[(d, v)] = z
                    
            # n_d_kのサンプル合計数をプラス
            sum_ndz += self.ndz
            # n_d_kのサンプル平均を計算
            ave_ndz = sum_ndz / (s+1)
            self.__update_dirichlet_param(ave_ndz, n_docs)
            elapsed_time = time.time() - start
            self.logger.info('1 iter time: {:.3f} [sec]'.format(elapsed_time))
            
            if (s+1) % self.PRINT_EVERY == 0:
                print('{} iter finished !'.format(s+1))
                if X_test:
                    print('lik : {}'.format(self.perplexity(X_test)))
        return self
    
    
    def perplexity(self, X_test):
        test_n_docs, n_features = X_test.shape
        
        indices = X_test.indices
        indptr = X_test.indptr
        data = X_test.data
        
        lik = 0.0
        self.sum_alpha = self.alpha.sum()
        right_denom = self.nz + self.sum_beta
        self.logger.info('test_n_docs: {}'.format(test_n_docs))
        self.logger.info('indices \n{}'.format(indices))
        self.logger.info('indptr \n{}'.format(indptr))
        self.logger.info('data \n{}'.format(data))
        sum_nd = 0
        for d in range(test_n_docs):
            vs = indices[indptr[d]:indptr[d+1]]
            left = (self.ndz[d]+self.alpha)\
                    / (self.nd[d] + self.sum_alpha)
            n_vs = data[indptr[d]:indptr[d+1]]
            for v, n_v in zip(vs, n_vs):
            #for v in vs:
                pv = (self.nzv[:, v]+self.beta[0])\
                        * left / right_denom
                lik += np.log(pv.sum())
                sum_nd += n_v
        
        return np.exp(-lik / sum_nd)
    
    def print_topn_words(self, n, id2word):
        index = np.arange(n) + 1
        df = pd.DataFrame(data=[], index=index)
        for z in range(self.n_topics):
            idx_descend = self.nzv[z].argsort()[::-1]
            top_n = [id2word[idx] for idx in idx_descend[:n]]
            df['topic{}'.format(z)] = top_n
        
        display(df)
        

def split_data(X, ratio=.7):
    """
    per-doc split to train_words & test_words  ratio : 1-ratio 
    """
    from scipy.sparse import csr_matrix
    
    n_docs, n_features = X.shape
    #train_n_features = int(np.round(n_features * ratio))
    
    indices = X.indices
    indptr = X.indptr
    data = X.data
    
    train_indptr = np.zeros(len(indptr))
    test_indptr = np.zeros(len(indptr))
    train_indices = np.array([],dtype=np.int32)
    test_indices = np.array([],dtype=np.int32)
    train_data = np.array([],dtype=np.int32)
    test_data = np.array([],dtype=np.int32)
    
    for d in range(n_docs):
        vs = indices[indptr[d]:indptr[d+1]]
        n_vs = data[indptr[d]:indptr[d+1]]
        
        idxs = np.arange(len(vs))
        np.random.shuffle(idxs)
        train_n_features = int(np.round(len(vs) * ratio))
        train_indices = np.hstack((train_indices,vs[idxs[:train_n_features]]))
        test_indices = np.hstack((test_indices,vs[idxs[train_n_features:]]))
        
        train_data = np.hstack((train_data,n_vs[idxs[:train_n_features]]))
        test_data = np.hstack((test_data,n_vs[idxs[train_n_features:]]))
        
        train_indptr[d+1] = len(train_indices)
        test_indptr[d+1] = len(test_indices)
    
    X_train = csr_matrix((train_data, train_indices, train_indptr),
                         shape=(n_docs, n_features))
    X_test = csr_matrix((test_data, test_indices, test_indptr),
                         shape=(n_docs, n_features))
    
    return X_train, X_test
    

def main():
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger(__name__)
    
    X_train, X_test = split_data(X)
    n_docs, n_features = X.shape
    lda = LDA(n_iter=25, n_topics=10, logger=logger)
    lda.fit(X)
    lda.print_topn_words(n=10, id2word=id2word)
    
main()

INFO:__main__:ndz: (2700, 10), nzv: (10, 2432)
INFO:__main__:iteration start!
INFO:__main__:1 iter time: 4.021 [sec]
INFO:__main__:1 iter time: 4.002 [sec]
INFO:__main__:1 iter time: 4.100 [sec]
INFO:__main__:1 iter time: 4.047 [sec]
INFO:__main__:1 iter time: 4.019 [sec]
INFO:__main__:1 iter time: 4.057 [sec]
INFO:__main__:1 iter time: 4.515 [sec]
INFO:__main__:1 iter time: 4.991 [sec]
INFO:__main__:1 iter time: 4.426 [sec]
INFO:__main__:1 iter time: 4.086 [sec]
INFO:__main__:1 iter time: 4.011 [sec]
INFO:__main__:1 iter time: 4.038 [sec]
INFO:__main__:1 iter time: 4.011 [sec]
INFO:__main__:1 iter time: 4.208 [sec]
INFO:__main__:1 iter time: 4.037 [sec]
INFO:__main__:1 iter time: 3.985 [sec]
INFO:__main__:1 iter time: 3.989 [sec]
INFO:__main__:1 iter time: 4.047 [sec]
INFO:__main__:1 iter time: 4.000 [sec]
INFO:__main__:1 iter time: 4.018 [sec]


20 iter finished !


INFO:__main__:1 iter time: 4.472 [sec]
INFO:__main__:1 iter time: 4.253 [sec]
INFO:__main__:1 iter time: 4.139 [sec]
INFO:__main__:1 iter time: 4.255 [sec]
INFO:__main__:1 iter time: 4.341 [sec]


,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
1,さん,映画,ゴルフ,更新,通,さん,対応,年,転職,話題
2,女性,公開,氏,ソフトウェア,アプリ,自分,搭載,氏,話題,円
3,写真,作品,位,画面,機能,歳,D,もの,情報,肌
4,もの,月日,選手,アプリ,DEJI,女性,発売,日本,万,月日
5,円,年,年,利用,デジタル,結婚,発表,それ,自分,ため
6,そう,監督,SportsWatch,S,ゲーム,私,バッテリー,的,円,iPad
7,店,日本,番組,表示,パソコン,いい,smartphone,年収,livedoor,使用
8,方,本作,放送,smartphone,対応,男性,充電,ため,仕事,チェック
9,女子,登場,これ,エスマックス,ため,それ,万,方,Twitter,登場
10,クリスマス,世界,時,Android,サービス,彼,モデル,これ,ユーザー,人気


## sklearn's model


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import time


lda_sklearn = LatentDirichletAllocation(n_components=10, learning_method='batch',
                                max_iter=5, random_state=0)

start = time.time()
document_topics = lda_sklearn.fit_transform(X)
elapsed_time = time.time() - start
print(elapsed_time)

sorting = np.argsort(lda_sklearn.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names())
mglearn.tools.print_topics(topics=range(10), feature_names=feature_names,
                          sorting=sorting, topics_per_chunk=5, n_words=10)

In [25]:
import time
n_topics = 20
n_docs = 2700
n_features = 10000
z = np.random.randint(0, n_topics, size=(n_docs,n_features))
start = time.time()
n_dk = np.zeros((n_docs,n_topics))
n_kv = np.zeros((n_topics,n_features))
for d, (z_d) in enumerate(z):
    n_dk[d] += np.bincount(z_d, minlength=n_topics)
for i in range(n_features):
    n_kv[:, i] = np.bincount(z[:,i], minlength=n_topics)
elapsed_time = time.time() - start
print(elapsed_time)

start = time.time()
n_dk_along = np.apply_along_axis(lambda x: np.bincount(x, minlength=n_topics),
                                 axis=1,
                                 arr=z)
n_kv_along = np.apply_along_axis(lambda x: np.bincount(x, minlength=n_topics),
                                 axis=0,
                                 arr=z)
elapsed_time = time.time() - start
print('numpy along: {} [sec]'.format(elapsed_time))
print(n_kv.sum(axis=1))

0.39963197708129883
numpy along: 0.4303920269012451 [sec]
[ 1351895.  1348678.  1349448.  1350742.  1350503.  1350522.  1351071.
  1348535.  1349542.  1348857.  1349303.  1349917.  1349441.  1350625.
  1350604.  1350020.  1350183.  1349080.  1349876.  1351158.]


In [28]:
import scipy.special as special

vs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
vs_index = np.arange(len(vs))

ratio = .7
n_features = int(np.round(len(vs)*ratio))

np.random.shuffle(vs_index)
print(vs_index[:n_features])
print(vs_index[n_features:])
print(vs[vs_index[:n_features]])
print(vs[vs_index[n_features:]])

train_indices = np.array([1, 2, 3], dtype=np.int32)
train_indices = np.hstack((train_indices, vs[vs_index[:n_features]]))
print(train_indices)

[3 5 6 8 4 1 9]
[2 0 7]
[ 4  6  7  9  5  2 10]
[3 1 8]
[ 1  2  3  4  6  7  9  5  2 10]


In [28]:
a = np.array([[1, 2, 3, 4, 5],
              [6, 7, 8, 9, 10],
              [11, 12, 13, 14, 15],
              [16, 17, 18, 19, 20],
              [21, 22, 23, 24, 25]], dtype=np.float32)
columns = [['a', 'b', 'c', 'd', 'e'],
           ['f', 'g', 'h', 'i', 'j']]
topic = np.arange(10) * 10

index = np.arange(10) + 1
df = pd.DataFrame(data=[], index=index)
df['topic1'] = topic
display(df)

,topic1
1,0
2,10
3,20
4,30
5,40
6,50
7,60
8,70
9,80
10,90


In [52]:
print(X.shape)
print(id2word)


(2700, 2432)
{0: ' 新モデル', 1: '.com', 2: '.jp', 3: '1seg', 4: 'A', 5: 'AKB', 6: 'ANDROID', 7: 'Amazon', 8: 'Amazon.co.jp', 9: 'Android', 10: 'Androidアプリ', 11: 'Apple', 12: 'B', 13: 'Blu-Ray', 14: 'Bluetooth', 15: 'C', 16: 'CM', 17: 'CMOS', 18: 'CPU', 19: 'CROWD', 20: 'Cochira', 21: 'D', 22: 'DEJI', 23: 'DETAILS', 24: 'DLNA', 25: 'DVD', 26: 'DoCoMo', 27: 'Dual Core', 28: 'E', 29: 'F', 30: 'FFPTSGSDHCE', 31: 'FOMA', 32: 'Facebook', 33: 'Felica', 34: 'Folder', 35: 'G', 36: 'GB', 37: 'GBClass', 38: 'GHz', 39: 'GPS', 40: 'GSM', 41: 'Google', 42: 'GooglePlay', 43: 'GooglePlayStore', 44: 'HD', 45: 'HDD', 46: 'HDMI', 47: 'HOMME', 48: 'HP', 49: 'HTML', 50: 'HTTP', 51: 'I', 52: 'ICS', 53: 'ID', 54: 'IEEE', 55: 'IPX', 56: 'IT', 57: 'IceCreamSandwich', 58: 'K', 59: 'KDDI', 60: 'KOMATSU', 61: 'Kindle', 62: 'L', 63: 'LED', 64: 'LTE', 65: 'M', 66: 'MAX', 67: 'MAXsmaxjponTwitter', 68: 'MB', 69: 'MHL', 70: 'MIYUKI', 71: 'MOVIEENTER', 72: 'MP', 73: 'MSM', 74: 'Mac', 75: 'Mbps', 76: 'N', 77: 'NHK', 78: 'N